## Imports

In [1]:
import os

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/__init__.py:22: UserWarning: ALESPICEROOT environment variable not set, Spice Drivers will not function correctly
  warnings.warn('ALESPICEROOT environment variable not set, Spice Drivers will not function correctly')


## Load Network and Generate Sensors

In [2]:
cubes = '/scratch/csm2020/data/cubes2.lis'
sensors = generate_sensors(cubes)

network = '/scratch/csm2020/data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors)

Trying <class 'ale.drivers.dawn_drivers.DawnFcPds3NaifSpiceDriver'>
Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.mex_drivers.MexHrscIsisLabelNaifSpiceDriver'>
Failed: Instrument ID is wrong.

Trying <class 'ale.drivers.mex_drivers.MexHrscPds3NaifSpiceDriver'>


Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/dawn_drivers.py", line 37, in instrument_id
    instrument_id = super().instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/base/label_pds3.py", line 29, in instrument_id
    return self.label['INSTRUMENT_ID']
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/pvl/_collections.py", line 123, in __getitem__
    return dict_getitem(self, key)[0]
KeyError: 'INSTRUMENT_ID'
Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/mex_drivers.py", line 497, in instrument_id
    raise Exception

Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.mex_drivers.MexSrcPds3NaifSpiceDriver'>
Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.selene_drivers.KaguyaMiPds3NaifSpiceDriver'>
Failed: None

Trying <class 'ale.drivers.selene_drivers.KaguyaTcPds3NaifSpiceDriver'>
Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.mro_drivers.MroCtxIsisLabelNaifSpiceDriver'>


Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/mex_drivers.py", line 170, in instrument_id
    if(super().instrument_id != "HRSC"):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/base/label_pds3.py", line 29, in instrument_id
    return self.label['INSTRUMENT_ID']
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/pvl/_collections.py", line 123, in __getitem__
    return dict_getitem(self, key)[0]
KeyError: 'INSTRUMENT_ID'
Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/mex_drivers.py", line 648, in instrument_id
    if(super().instr

Failed: ale.spice_root is not set, cannot search for metakernels. ale.spice_root = "None"

Trying <class 'ale.drivers.mro_drivers.MroCtxPds3LabelNaifSpiceDriver'>
Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.viking_drivers.VikingIsisLabelNaifSpiceDriver'>
Failed: ale.spice_root is not set, cannot search for metakernels. ale.spice_root = "None"

Trying <class 'ale.drivers.co_drivers.CassiniIssPds3LabelNaifSpiceDriver'>
Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.nh_drivers.NewHorizonsLorriIsisLabelNaifSpiceDriver'>


...

Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/co_drivers.py", line 126, in instrument_id
    return self.id_lookup[super().instrument_id]
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/base/label_pds3.py", line 29, in instrument_id
    return self.label['INSTRUMENT_ID']
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/pvl/_collections.py", line 123, in __getitem__
    return dict_getitem(self, key)[0]
KeyError: 'INSTRUMENT_ID'
Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/nh_drivers.py", line 33, in instrument_id
    return id_l

Failed: 'Mars_Reconnaissance_Orbiter'

Trying <class 'ale.drivers.mess_drivers.MessengerMdisIsisLabelNaifSpiceDriver'>
Failed: 'CTX'

Trying <class 'ale.drivers.mess_drivers.MessengerMdisPds3NaifSpiceDriver'>
Failed: 'INSTRUMENT_ID'

Trying <class 'ale.drivers.selene_drivers.KaguyaTcIsisLabelIsisSpiceDriver'>


Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/mess_drivers.py", line 288, in instrument_id
    return ID_LOOKUP[super().instrument_id]
KeyError: 'CTX'
Traceback (most recent call last):
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/__init__.py", line 77, in load
    res.instrument_id
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/drivers/mess_drivers.py", line 132, in instrument_id
    return ID_LOOKUP[super().instrument_id]
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/ale/base/label_pds3.py", line 29, in instrument_id
    return self.label['INSTRUMENT_ID']
  File "/home/kberry/anaconda3/envs/knoten/lib/python3.7/site-packages/pvl/_collections.py", line 123, in __getitem__
    return dict_get

Failed: 'InstrumentName'

Trying <class 'ale.drivers.mro_drivers.MroCtxIsisLabelIsisSpiceDriver'>
Success with:  <ale.drivers.mro_drivers.MroCtxIsisLabelIsisSpiceDriver object at 0x7fc7b5c915d0>
ISD:
 {
  "radii": {
    "semimajor": 3396.19,
    "semiminor": 3376.2,
    "unit": "km"
  },
  "sensor_position": {
    "positions": [
      [
        3251057.7839916283,
        -1147253.4509119922,
        1272525.1710562017
      ],
      [
        3250910.2196090613,
        -1147253.746541773,
        1272910.4708803867
      ],
      [
        3250762.6147560426,
        -1147254.0253945119,
        1273295.7549033235
      ],
      [
        3250614.96958111,
        -1147254.2874693046,
        1273681.0227384467
      ],
      [
        3250467.2840134143,
        -1147254.5327656665,
        1274066.2745718886
      ],
      [
        3250319.558055226,
        -1147254.7612829695,
        1274451.5103988778
      ],
      [
        3250171.7916355934,
        -1147254.973020687,
   

Success!


## Determine Which Sensor Parameters to Solve For

In [3]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
for sn, parameters in all_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

Image: MRO/CTX/1085197697:073
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1096561308:045
  IT Pos. Bias    | 0 | 0.0
  CT Pos. Bias    | 1 | 0.0
  Rad Pos. Bias   | 2 | 0.0
  IT Vel. Bias    | 3 | 0.0
  CT Vel. Bias    | 4 | 0.0
  Rad Vel. Bias   | 5 | 0.0
  Omega Bias      | 6 | 0.0
  Phi Bias        | 7 | 0.0
  Kappa Bias      | 8 | 0.0
  Omega Rate      | 9 | 0.0
  Phi Rate        | 10 | 0.0
  Kappa Rate      | 11 | 0.0
  Omega Accl      | 12 | 0.0
  Phi Accl        | 13 | 0.0
  Kappa Accl      | 14 | 0.0
  Focal Bias      | 15 | 0.0
Image: MRO/CTX/1136952576:186
  

## Solve for angles and angular rates

In [4]:
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

## Compute the Column Indices for Parameters

In [5]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [6]:
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)

## Whole (Dense) Bundle Process in a loop with LM terms

The next three cells show examples of the bundle adjustment loop running with Levenberg-Marquat added.

This example shows the first heuristic for initializing $\lambda$, the weighting parameter, from Numerical Recipes in C++

NumericalRecipes in C++ recommended starting with $\lambda$ = 0.01, and then increasing or decreasing this value by an order of magnitude each iteration depending on whether the last iteration increased or decreased 
$\sigma_0$.

Step sizes will get smaller as the target is approached.

### LM using Numerical Recipies's heuristic

$\lambda = 0.01$ to start out, then increased or decreased by an order of magnitude each step. 

`damping` is used as the variable for $\lambda$, since in python `lambda` is a reserved word.

In [10]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 100
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 0.01
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)    
#    C = J.T.dot(W_observations).dot(V) - damping*np.identity(W_params.shape[0]).dot(total_correction) - W_params.dot(total_correction)
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction)
    N = J.T.dot(W_observations).dot(J) + damping*np.identity(W_params.shape[0]) + W_params
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    
    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    if (sigma0 < old_sigma0):
        damping*=0.1
    else:
        damping*=10
        
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

iteration 0: sigma0 = 0.6091559677586023

corrections: mean = 0.0050501668341782924 min = -2.929367872277257 max = 6.343296369797652
iteration 1: sigma0 = 0.44571672510323584

corrections: mean = 0.0005553105290355316 min = -0.7217554362822134 max = 0.80518818248211
iteration 2: sigma0 = 0.42294889581955025

corrections: mean = 8.78879236274064e-06 min = -0.021506405844758915 max = 0.024082007707184407
iteration 3: sigma0 = 0.422453244846492

corrections: mean = 7.622034105195536e-09 min = -6.387263193818047e-05 max = 7.043873999056952e-05
iteration 4: sigma0 = 0.4224526642405469

corrections: mean = -2.5049903991523305e-11 min = -1.7404638515765472e-08 max = 1.622475249539117e-08
iteration 5: sigma0 = 0.4224526641633733

change in sigma0 of 7.717360084313896e-11 converged!


### sba/levmar's heuristic

$\lambda$ initially set to a user supplied value (or default = 0.001) * the max value in $J^{T}J$

Updates either:

(1) $\lambda = max(1/3, 1-(2\rho -1)^3) , \nu = 2.0$

(2) $\lambda = \lambda\nu, \nu = 2.0\nu$

See: http://users.ics.forth.gr/~argyros/mypapers/2004_08_tr340_forth_sba.pdf

In [14]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 25
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 0.001
nu = 2.0
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)
    if i==0:
        damping *= np.max(J.T.dot(J))
    
#    C = J.T.dot(W_observations).dot(V) - damping*np.identity(W_params.shape[0]).dot(total_correction) - W_params.dot(total_correction)
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction)
    N = J.T.dot(W_observations).dot(J) + damping*np.identity(W_params.shape[0]) + W_params
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    
    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    old_V = V
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    if (sigma0 < old_sigma0):
        damping = max(1.0/3.0, (np.linalg.norm(old_V)**2 - np.linalg.norm(V)**2)/(dX.T.dot(damping*dX + C)))
        nu = 2
    else:
        damping = damping * nu
        nu = 2 * nu

    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

iteration 0: sigma0 = 0.38654551169484236

corrections: mean = 9.207596857798319e-05 min = -0.5693190902275064 max = 0.6292929755622529
iteration 1: sigma0 = 0.38564887830266575

corrections: mean = 2.2990661839429544e-07 min = -0.0005420319875632349 max = 0.0005881940438031024
iteration 2: sigma0 = 0.38519350821896375

corrections: mean = 1.7365425206931952e-08 min = -4.2302950725249885e-05 max = 4.60299555500806e-05
iteration 3: sigma0 = 0.3851934382244883

corrections: mean = 1.6462819296938826e-08 min = -4.1512960985117e-05 max = 4.5282724710033444e-05
iteration 4: sigma0 = 0.3851933698958462

corrections: mean = 1.580169589302863e-08 min = -4.13043737341095e-05 max = 4.515770251034494e-05
iteration 5: sigma0 = 0.3851933019111736

corrections: mean = 1.5153586807330953e-08 min = -4.1113861728498275e-05 max = 4.504296520344739e-05
iteration 6: sigma0 = 0.38519323422818896

corrections: mean = 1.4512239826812034e-08 min = -4.091736937490866e-05 max = 4.491304238949076e-05
iteration 7